In [1]:
import os
import sys
import pandas as pd
import requests
import re
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
import json

# 커뮤니티 사이트 크롤링

### dcinside

In [2]:
base_url = 'https://www.dcinside.com'

In [3]:
browser = Chrome()
browser.maximize_window()

In [10]:
browser.get(base_url)
browser.implicitly_wait(3)

In [11]:
browser.find_elements_by_xpath('//*[@class = "in_keyword"]')[0].click()

In [12]:
#갤러리 검색어 입력
browser.find_elements_by_xpath('//*[@class = "in_keyword"]')[0].send_keys('여자패션')

In [14]:
#갤러리명을 바로 클릭해서 갤러리로 이동
browser.find_elements_by_xpath('//*[@class = "auto_wordwrap"]/div[1]//ul/li/a')[0].click()

In [9]:
#돋보기 버튼 클릭
browser.find_elements_by_xpath('//*[@class = "sp_main bnt_search"]')[0].click()

In [15]:
#제어 브라우저의 소스코드 불러오기
html0 = browser.page_source

In [16]:
html1 = BeautifulSoup(html0,'html.parser')

In [18]:
page0 = html1.find('div',{'class':'bottom_paging_box'})

In [32]:
page0.find_all('a')[-1]['href']
#마지막 페이지가 25페이지 인 것을 알 수 있음

'/mgallery/board/lists/?id=girlsfashion&page=25'

In [26]:
gall_url = 'http://gall.dcinside.com'

In [28]:
page_url = gall_url + '/mgallery/board/lists/?id=girlsfashion&page=' + '{}'

In [38]:
req = requests.get(page_url.format(str(1)))

In [40]:
soup = BeautifulSoup(req.content,'html.parser')

In [43]:
posts0 = soup.find('table',{'class':'gall_list'}).find('tbody')

In [45]:
posts = posts0.find_all('tr',{'class':'ub-content us-post'})

In [49]:
post_title = posts[0].find('td',{'class':'gall_tit ub-word'}).find('a').text

'차정원 모한원피스 예쁘다'

In [50]:
post_date = posts[0].find('td',{'class':'gall_date'})['title']

In [52]:
post_count = posts[0].find('td',{'class':'gall_count'}).text

In [58]:
post_url = gall_url+posts[0].find('td',{'class':'gall_tit ub-word'}).find('a')['href']

'http://gall.dcinside.com/mgallery/board/view/?id=girlsfashion&no=1941&page=1'

In [96]:
fashion_data = pd.DataFrame({'post_title':[],
                            'post_date':[],
                            'post_count':[],
                            'post_url':[]})
for i in range(1,11):
    req2 = requests.get(page_url.format(str(i)))
    soup2 = BeautifulSoup(req2.content,'html.parser')
    
    posts0 = soup.find('table',{'class':'gall_list'}).find('tbody')
    posts = posts0.find_all('tr',{'class':'ub-content us-post'})
    for j in range(len(posts)):
        post_title = posts[j].find('td',{'class':'gall_tit ub-word'}).find('a').text
        post_date = posts[j].find('td',{'class':'gall_date'})['title']
        post_count = posts[j].find('td',{'class':'gall_count'}).text
        post_url = post_url = gall_url+posts[j].find('td',{'class':'gall_tit ub-word'}).find('a')['href']
        
        insert_data = pd.DataFrame({'post_title':[post_title],
                            'post_date':[post_date],
                            'post_count':[post_count],
                            'post_url':[post_url]})
        fashion_data = fashion_data.append(insert_data)
fashion_data.index = range(len(fashion_data))
        

In [97]:
fashion_data

,post_title,post_date,post_count,post_url
0,차정원 모한원피스 예쁘다,2019-04-16 10:24:03,3,http://gall.dcinside.com/mgallery/board/view/?...
1,브리 라슨 존예 존예,2019-04-16 04:51:11,11,http://gall.dcinside.com/mgallery/board/view/?...
2,캡틴마블 브리라슨 튀는 패션 존예,2019-04-15 19:16:25,9,http://gall.dcinside.com/mgallery/board/view/?...
3,유니섹스 스트릿,2019-04-14 10:08:35,10,http://gall.dcinside.com/mgallery/board/view/?...
4,아이린 오늘 스타일 노랑,2019-04-13 18:56:17,15,http://gall.dcinside.com/mgallery/board/view/?...
5,"오늘, 들꽃 영화상 스타 패션 ㄱㄱ",2019-04-13 13:35:34,27,http://gall.dcinside.com/mgallery/board/view/?...
6,어벤져스_엔드게임 캡틴마블 오늘 입국 패션 튀네,2019-04-13 11:59:05,11,http://gall.dcinside.com/mgallery/board/view/?...
7,경리 패션 눈 쏠림,2019-04-12 21:54:52,31,http://gall.dcinside.com/mgallery/board/view/?...
8,정려원 공항패션 >>,2019-04-12 20:40:23,23,http://gall.dcinside.com/mgallery/board/view/?...
9,이거 옷 잘입은거야?,2019-04-12 14:54:36,31,http://gall.dcinside.com/mgallery/board/view/?...


#### 게시물 본문 긁어오기

In [80]:
fashion_data['post_main_text'] = 1

In [82]:
fashion_data['post_main_text'].iloc[0] = post_text

C:\Users\tlsal\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [86]:
fashion_data

,post_title,post_date,post_count,post_url
0,차정원 모한원피스 예쁘다,2019-04-16 10:24:03,3,http://gall.dcinside.com/mgallery/board/view/?...
1,브리 라슨 존예 존예,2019-04-16 04:51:11,11,http://gall.dcinside.com/mgallery/board/view/?...
2,캡틴마블 브리라슨 튀는 패션 존예,2019-04-15 19:16:25,9,http://gall.dcinside.com/mgallery/board/view/?...
3,유니섹스 스트릿,2019-04-14 10:08:35,10,http://gall.dcinside.com/mgallery/board/view/?...
4,아이린 오늘 스타일 노랑,2019-04-13 18:56:17,15,http://gall.dcinside.com/mgallery/board/view/?...
5,"오늘, 들꽃 영화상 스타 패션 ㄱㄱ",2019-04-13 13:35:34,27,http://gall.dcinside.com/mgallery/board/view/?...
6,어벤져스_엔드게임 캡틴마블 오늘 입국 패션 튀네,2019-04-13 11:59:05,11,http://gall.dcinside.com/mgallery/board/view/?...
7,경리 패션 눈 쏠림,2019-04-12 21:54:52,31,http://gall.dcinside.com/mgallery/board/view/?...
8,정려원 공항패션 >>,2019-04-12 20:40:23,23,http://gall.dcinside.com/mgallery/board/view/?...
9,이거 옷 잘입은거야?,2019-04-12 14:54:36,31,http://gall.dcinside.com/mgallery/board/view/?...


In [90]:
aa = fashion_data['post_url'].iloc[0]

In [91]:
req3 = requests.get(aa)
soup3 = BeautifulSoup(req3.content,'html.parser')

In [92]:
post_text0 = soup3.find('div',{'class':'writing_view_box'}).get_text()

In [93]:
post_text0

'\n \n차정원이 입은 원피스 넘 예뻐서 찾아보는데 w컨셉에서 팔고있네요모한원피스 아이보리 대존예인듯스페이스4 할인하고 있어서 바로 질러버렸어요ㅎㅎㅎㅎㅎㅎ \n'

In [94]:
post_text = re.sub('\n','',post_text0)

In [95]:
post_text

' 차정원이 입은 원피스 넘 예뻐서 찾아보는데 w컨셉에서 팔고있네요모한원피스 아이보리 대존예인듯스페이스4 할인하고 있어서 바로 질러버렸어요ㅎㅎㅎㅎㅎㅎ '

In [109]:
#지금 포스트에 들어가서 소스코드가 안불러와지므로 일단 보류
#코드상 문제 x
fashion_data2 = pd.DataFrame({'post_title':[],
                            'post_date':[],
                            'post_count':[],
                            'post_url':[],
                            'post_main_text':[]})

for i in range(len(fashion_data)): 
    aa = fashion_data['post_url'].iloc[i]
    post_title = fashion_data['post_title'].iloc[i]
    post_date  = fashion_data['post_date'].iloc[i]
    post_count = fashion_data['post_count'].iloc[i]
    
    req3 = requests.get(aa)
    soup3 = BeautifulSoup(req3.content,'html.parser')
    
    try:
        post_text0 = soup3.find('div',{'class':'writing_view_box'}).get_text()
        post_text =  re.sub('\n','',post_text0)
        
        insert_data = pd.DataFrame({'post_title':[post_title],
                                    'post_date':[post_date],
                                    'post_count':[post_count],
                                    'post_url':[aa],
                                    'post_main_text':[post_text0]})
        fashion_data2 = fashion_data2.append(insert_data)
    except:
        post_text = ''
        insert_data = pd.DataFrame({'post_title':[post_title],
                                    'post_date':[post_date],
                                    'post_count':[post_count],
                                    'post_url':[aa],
                                    'post_main_text':[post_text]})
        fashion_data2 = fashion_data2.append(insert_data)
    
fashion_data2.index = range(len(fashion_data2))

In [114]:
fashion_data2

,post_title,post_date,post_count,post_url,post_main_text
0,차정원 모한원피스 예쁘다,2019-04-16 10:24:03,3,http://gall.dcinside.com/mgallery/board/view/?...,
1,브리 라슨 존예 존예,2019-04-16 04:51:11,11,http://gall.dcinside.com/mgallery/board/view/?...,
2,캡틴마블 브리라슨 튀는 패션 존예,2019-04-15 19:16:25,9,http://gall.dcinside.com/mgallery/board/view/?...,
3,유니섹스 스트릿,2019-04-14 10:08:35,10,http://gall.dcinside.com/mgallery/board/view/?...,
4,아이린 오늘 스타일 노랑,2019-04-13 18:56:17,15,http://gall.dcinside.com/mgallery/board/view/?...,
5,"오늘, 들꽃 영화상 스타 패션 ㄱㄱ",2019-04-13 13:35:34,27,http://gall.dcinside.com/mgallery/board/view/?...,
6,어벤져스_엔드게임 캡틴마블 오늘 입국 패션 튀네,2019-04-13 11:59:05,11,http://gall.dcinside.com/mgallery/board/view/?...,
7,경리 패션 눈 쏠림,2019-04-12 21:54:52,31,http://gall.dcinside.com/mgallery/board/view/?...,
8,정려원 공항패션 >>,2019-04-12 20:40:23,23,http://gall.dcinside.com/mgallery/board/view/?...,
9,이거 옷 잘입은거야?,2019-04-12 14:54:36,31,http://gall.dcinside.com/mgallery/board/view/?...,


### 네이트판

#### 셀레늄으로 크롤링을 하면 페이지 수 끝까지 데이터를 수집할 수 있지만 일단 여기서는 10페이지까지 데이터를 수집하는 것을 목표

In [115]:
base_url = 'https://pann.nate.com'

In [116]:
browser.get(base_url)

In [127]:
browser.find_elements_by_xpath('//*[@id = "searchWrap"]/form[3]/fieldset//div/input')[0].click()

In [128]:
#원하는 검색어 입력
browser.find_elements_by_xpath('//*[@id = "searchWrap"]/form[3]/fieldset//div/input')[0].send_keys('프로농구')

In [130]:
#경로지정시에 /와 //의 구분을 명확히 하자 -> /는 부모태그밑의 요소들을 선택할 때고 //는 바로 밑이 아닌 그 다음
#경로로 이동시에 사용
browser.find_elements_by_xpath('//*[@id = "searchWrap"]/form[3]/fieldset/input[1]')[0].click()

In [135]:
#최종 판목록으로 이동
browser.find_elements_by_xpath('//*[@class = "srcharea"]/div[3]/p/a')[0].click()

In [ ]:
nate_page = 'https://pann.nate.com/search/talk?q=프로농구&page=' + '{}'

In [136]:
nate_html  = browser.page_source

In [138]:
nate_html2 = BeautifulSoup(nate_html,'html.parser')

In [143]:
posts0 = nate_html2.find('div',{'class':'srch_list section'}).find('ul',{'class':'s_list'})

In [ ]:
nate_page = 'https://pann.nate.com/search/talk?q=프로농구&page=' + '{}'

In [145]:
posts = posts0.find_all('li')

In [153]:
post_url = base_url+posts[0].find('dt').find('a')['href'] #게시판 url

In [154]:
req = requests.get(post_url)
soup = BeautifulSoup(req.content,'html.parser')

In [155]:
soup.find('div',{'class':'post-tit-info'}).find('h4').text#게시판 제목

'휴식기 끝낸 프로농구 내일부터 시즌 재계'

In [157]:
soup.find('div',{'class':'post-tit-info'}).find('div',{'class':'info'}).find('span').text#날짜

'2018.12.05 17:19'

In [158]:
#조회수
soup.find('div',{'class':'post-tit-info'}).find('div',{'class':'info'}).find('span',{'class':'count'}).text

'조회10'

In [159]:
#게시판 내용
soup.find('div',{'class':'posting'}).find('div',{'id':'espresso_editor_view'}).get_text()

'국가대표 경기로 휴식기를 가졌던 2018-2019 SKT 5GX 프로농구가 돌아온다. 프로농구는 지난달 25일 이후 2019 국제농구연맹 월드컵 아시아 오세아니아 지역 예선 2경기를 위해 휴식기에 돌입했다. 약 10일간 재정비의 시간을 가졌던 구단들은 6일 전자랜드와 서울SK, DB와 오리온 경기부터 3라운드를 시작한다. 휴식기를 맞아 구단들은 부진한 외국인선수들을 적극 교체하는 등 전력 업그레이드를 타진했다. 또 3라운드부터는 지난달 26일 열린 신인 드래프트에서 선발된 선수들이 경기에 출전 가능하기 때문에 신인들은 즉시 전력으로 활용하기 위해 손발을 맞췄다.\xa0시즌 개막에 앞서 ‘4강 후보’로 지목된 안양 KGC인삼공사는 휴식기에 외국인 선수 2명을 모두 바꿨다. 인삼공사는 장신 외국인 선수인 미카일 매킨토시와 계약을 해지하고 지난 시즌 울산 현대모비스에서 평균 22.2점에 7.3리바운드를 기록한 레이션 테리를 영입했다. 또 햄스트링 부상으로 시즌 아웃된 랜디 컬페퍼를 저스틴 에드워즈로 교체했다. 익스플로러 보러가기인삼공사는 신인도 전체 2순위 가드 변준형을 선발한데다 부산 kt와 트레이드를 통해 가드 박지훈까지 영입하며 약점으로 지적받던 가드진을 보강했다. \xa0서울 SK는 단신 외국인 선수를 오데리언 바셋에서 마커스 쏜튼으로 교체했다. 바셋은 올 시즌 SK에서 평균 15.1득점, 3.3도움으로 기대 이하의 활약으로 결별하게 됐다. 쏜튼은 2015 미국프로농구(NBA) 드래프트에서 전체 45순위로 보스턴 셀틱스의 지명을 받았고, 이후 호주, 이탈리아, 터키 리그 등에서 선수 생활을 했다.\xa0최근 7연패를 당하며 최하위에 처진 서울 삼성은 글렌 코지 대신 네이트 밀러를 영입, 부진 탈출을 위한 승부수를 띄웠다.\n'

In [165]:
nate_data2 = pd.DataFrame({'post_title':[],
                            'post_date':[],
                            'post_count':[],
                            'post_url':[],
                            'post_main_text':[]})
for i in range(1,11):
    try:
        res = requests.get(nate_page.format(str(i)))
        soup = BeautifulSoup(res.content,'html.parser')

        posts0 = soup.find('div',{'class':'srch_list section'}).find('ul',{'class':'s_list'})
        posts = posts0.find_all('li')

        for j in range(len(posts)):

            post_url = base_url+posts[j].find('dt').find('a')['href'] #게시판 url

            req2 = requests.get(post_url)
            soup2 = BeautifulSoup(req2.content,'html.parser')

            post_title = soup2.find('div',{'class':'post-tit-info'}).find('h4').text#게시판 제목
            post_date = soup2.find('div',{'class':'post-tit-info'}).find('div',{'class':'info'}).find('span').text#날짜
            post_count = soup2.find('div',{'class':'post-tit-info'}).find('div',{'class':'info'}).find('span',{'class':'count'}).text
            post_main_text = soup2.find('div',{'class':'posting'}).find('div',{'id':'espresso_editor_view'}).get_text()

            insert_data = pd.DataFrame({'post_title':[post_title],
                                        'post_date':[post_date],
                                        'post_count':[post_count],
                                        'post_url':[post_url],
                                        'post_main_text':[post_main_text]})
            nate_data2 = nate_data2.append(insert_data)
    except:
        pass
    

In [166]:
nate_data2

,post_title,post_date,post_count,post_url,post_main_text
0,휴식기 끝낸 프로농구 내일부터 시즌 재계,2018.12.05 17:19,조회10,https://pann.nate.com/talk/344389852,국가대표 경기로 휴식기를 가졌던 2018-2019 SKT 5GX 프로농구가 돌아온다...
0,"프로농구 서울삼성 음발라, 펠프스로 교체",2018.11.13 18:41,조회4,https://pann.nate.com/talk/344159309,프로농구 서울 삼성썬더스는 12일 외인 빅맨 벤 음발라(23·196㎝)를 기타사유(...
0,프로농구 서울SK나이츠 홈개막전 승리,2018.10.13 16:54,조회19,https://pann.nate.com/talk/343817579,오늘낮 프로농구가 개막 했다 지난시즌 챔피언결정전에서 맞붙었던 두팀이 이번시즌 개막...
0,이번주 토요일 프로농구 정규시즌 개막,2018.10.10 18:16,조회13,https://pann.nate.com/talk/343782867,‘겨울스포츠의 대명사’ 프로농구가 6개월의 대장정의 막을 올린다.2018-2019...
0,프로농구 서울SK 대역전 우승까지 -1,2018.04.16 21:29,조회7,https://pann.nate.com/talk/341741799,오늘저녁 원주치악체육관에서열린 프로농구 7전4선승제 챔피언결정전5차전에서 원정팀 ...
0,프로농구 챔피언결정전2패 서울충격,2018.04.10 21:42,조회4,https://pann.nate.com/talk/341679362,오늘저녁 원주치악체육관에서열린 프로농구 챔피언결정전2차전에서도 1차전 3점차 분패에...
0,프로농구 서울SK나이츠 챔피언결정전진출,2018.04.04 21:51,조회7,https://pann.nate.com/talk/341614066,프로농구 서울SK나이츠가 전주KCC이지스를꺽고 챔피언결전전에 진출.4강플레이오프 5...
0,프로농구 서울SK나이츠 챔피언결정전진출눈앞,2018.03.31 17:07,조회3,https://pann.nate.com/talk/341569649,프로농구 서울SK나이츠가 오늘낮 서울 잠실학생체육관 홈에서 열린 4강플레이오프2차전...
0,프로농구 4강PO1차전 서울SK 선승!!,2018.03.29 21:33,조회3,https://pann.nate.com/talk/341551840,오늘저녁 서울 잠실학생체육관에서열린 프로농구 서울SK나이츠VS전주KCC 이지스의 4...
0,울산현대모비스 통합우승에 한발짝 다가서다,2019.04.13 18:37,조회4,https://pann.nate.com/talk/346135038,프로농구 울산현대모비스피버스가 챔프 1차전을 접수했다. 울산 현대모비스는 13일(...
